In [ ]:
from zipfile import ZipFile
file_name="/content/drive/My Drive/scens_detection/Test_zip.zip"
with ZipFile(file_name,'r') as zip:
     zip.extractall()
print('---Done')

In [ ]:
from zipfile import ZipFile
file_name="/content/drive/My Drive/scens_detection/Train_zip.zip"
with ZipFile(file_name,'r') as zip:
     zip.extractall()
print(' ---Done')

In [ ]:
#!pip install tensorflow==1.15.0
#!pip install keras==2.2.4
import tensorflow as tf
print('Tensorflow version ==',tf.__version__)
print('='*20)
import keras
print('keras version ==',keras.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
#sns.set_style('darkgrid')
plt.style.use('fivethirtyeight')
from sklearn.metrics import classification_report,confusion_matrix,plot_roc_curve
from sklearn.model_selection import train_test_split

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]
train_path = '/content/Train'
valid_path = '/content/Test'

In [ ]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
 # useful for getting number of classes
folders = glob('/content/Train/*')

In [ ]:
folders

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set))  

In [ ]:
epoches=range(1,11)

In [ ]:

train_acc=r.history['accuracy']
test_acc=r.history['val_accuracy']
train_loss=r.history['loss']
test_loss=r.history['val_loss']


In [ ]:
#plot
plt.style.use('fivethirtyeight')
plt.plot(epoches,train_loss,"r--")
plt.plot(epoches,test_loss,"b-")
plt.legend(["Traning Loss :","Test Loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.plot(epoches,train_acc,"r--")
plt.plot(epoches,test_acc,"b-")
plt.legend(["Traning acuracy :","Test accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Acuracy")
plt.show()

In [ ]:
import pandas as pd
result=pd.DataFrame({'traing loss':train_loss,'traing accuracy':train_acc,'test loss':test_loss,'test accuracy ':test_acc})

In [ ]:
#Result Dataframe 
result

In [ ]:
model.save('/content/drive/My Drive/scens_detection/model_1.h5')
#tf.keras.models.save_model(model,filepath='/content/drive/My Drive/mask_detection_vgg.h5')


In [ ]:
from glob import glob
from keras.preprocessing import image
images_path_indoor=glob('/content/Test/indoor/*')
images_path_outdoor=glob('/content/Test/outdoor/*')

In [ ]:
# Prediction for Mask found in testing dataset
prediction=[]
actual=[]
for i in range(len(images_path_indoor)):
  img = image.load_img(images_path_indoor[i],target_size=(224,224))
  img = np.asarray(img)
  class_=['indoor','outdoor']
  img = np.expand_dims(img, axis=0)
  output = model.predict(img)
  prediction.append(class_[output.argmax()])
  actual.append('indoor')


In [ ]:
# Prediction for No Mask found in testing dataset

for i in range(len(images_path_outdoor)):
  img = image.load_img(images_path_outdoor[i],target_size=(224,224))
  img = np.asarray(img)
  class_=['indoor','outdoor']
  img = np.expand_dims(img, axis=0)
  output = model.predict(img)
  prediction.append(class_[output.argmax()])
  actual.append('outdoor')

In [ ]:
print(classification_report(actual,prediction,target_names=class_))

In [ ]:
# classification report plot
cr=pd.DataFrame()
cr['Prcision']=[0.90,0.85,0.87,0.88]
cr['Recall']=[0.87,0.88,0.88,0.88]
cr['F1-score']=[0.88,0.86,0.88,0.88]
cr.index=['indoor','outdoor','macro avg','weigthed avg']
#plt.figure(figsize=(10,10))
sns.heatmap(cr,annot=True,fmt='.3f',vmin=0,vmax=1,linewidths=2,cmap='Blues')
plt.title('VGG16 Model--classification report')

plt.show()

In [ ]:
cm=confusion_matrix(actual,prediction)
cm=pd.DataFrame(cm)
cm.index=cm.columns=class_
sns.heatmap(cm,annot=True,fmt='.0f',vmin=0,vmax=140,linewidths=2,cmap='Blues')
plt.title('VGG16 Model--Confusion matrix')
plt.xlabel('predicted class')
plt.ylabel('actual class')
plt.show()

In [ ]:
for i in range(5):
  img = image.load_img(images_path_indoor[i],target_size=(224,224))
  img = np.asarray(img)
  plt.imshow(img)
  #class_=['Mask found','Mask not found']
  img = np.expand_dims(img, axis=0)
  #from keras.models import load_model
  #saved_model = load_model("vgg16_1.h5")
  output = model.predict(img)
  plt.title('Prediction = '+str(class_[output.argmax()]))
  plt.show()

In [ ]:
for i in range(5):
  img = image.load_img(images_path_outdoor[i],target_size=(224,224))
  img = np.asarray(img)
  plt.imshow(img)
  #class_=['indoor','outdoor']
  img = np.expand_dims(img, axis=0)
  #from keras.models import load_model
  #saved_model = load_model("vgg16_1.h5")
  output = model.predict(img)
  plt.title('Prediction = '+str(class_[output.argmax()]))
  plt.show()